# BERT

We shall implement BERT.  For this tutorial, you may want to first look at my Transformers tutorial to get a basic understanding of Transformers. 

For BERT, the main difference is on how we process the datasets, i.e., masking.   Aside from that, the backbone model is still the Transformers.

In [1]:
import math
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pickle

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
# torch.set_default_device(device)
print(device)

cuda


## 1. Data

For simplicity, we shall use very simple data like this.

In [3]:
import csv

raw_text = []
with open('./dataset/booksummaries.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for line in reader:
        raw_text.append(line)

In [4]:
raw_text[:2] # only last elements are the text we want

[['620',
  '/m/0hhy',
  'Animal Farm',
  'George Orwell',
  '1945-08-17',
  '{"/m/016lj8": "Roman \\u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children\'s literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}',
  ' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon

In [5]:
raw_text = []
with open('./dataset/booksummaries.txt', 'r', encoding='utf-8') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for line in reader:
        raw_text.append(line[-1])

In [6]:
raw_text[0]

' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself leader. N

In [7]:
char_len = 0
for i in raw_text:
    char_len += len(i)
char_len



41581453

In [8]:
len(raw_text)

16559

In [9]:
raw_text = raw_text[:100]

In [10]:
char_len = 0
for i in raw_text:
    char_len += len(i)
char_len



524213

In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")

c:\Users\sung2_8l7o06c\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# nlp.max_length = 1000000

In [13]:
sentences = []
for book in raw_text:
    doc = nlp(book)
    sentences.extend(list(doc.sents))

text = [x.text.lower() for x in sentences] #lower case
text = [re.sub("[.,!?\\-]=", '', x) for x in text] #clean all symbols

text[:10]

[" old major, the old boar on the manor farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'beasts of england'.",
 'when major dies, two young pigs, snowball and napoleon, assume command and turn his dream into a philosophy.',
 'the animals revolt and drive the drunken and irresponsible mr jones from the farm, renaming it "animal farm".',
 'they adopt seven commandments of animal-ism, the most important of which is, "all animals are equal".',
 'snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly.',
 'the pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health.',
 'napoleon takes the pups from the farm dogs and trains them privately.',
 'napoleon and snowball struggle for leadership.',
 'when snowball announces his plans to build a windmill, napoleon has his dogs chase snowball away

In [14]:
raw_text

[' Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, \'Beasts of England\'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself leader. 

In [15]:
word_count = 0
max_word_in_sentence = 0

for sentence in text:
    print(sentence, "_____")
    words = sentence.split()
    word_count += len(words)
    print(words)
    max_word_in_sentence = max(max_word_in_sentence, len(words))

print('\n')
print(word_count)
print(max_word_in_sentence)

 old major, the old boar on the manor farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'beasts of england'. _____
['old', 'major,', 'the', 'old', 'boar', 'on', 'the', 'manor', 'farm,', 'calls', 'the', 'animals', 'on', 'the', 'farm', 'for', 'a', 'meeting,', 'where', 'he', 'compares', 'the', 'humans', 'to', 'parasites', 'and', 'teaches', 'the', 'animals', 'a', 'revolutionary', 'song,', "'beasts", 'of', "england'."]
when major dies, two young pigs, snowball and napoleon, assume command and turn his dream into a philosophy. _____
['when', 'major', 'dies,', 'two', 'young', 'pigs,', 'snowball', 'and', 'napoleon,', 'assume', 'command', 'and', 'turn', 'his', 'dream', 'into', 'a', 'philosophy.']
the animals revolt and drive the drunken and irresponsible mr jones from the farm, renaming it "animal farm". _____
['the', 'animals', 'revolt', 'and', 'drive', 'the', 'drunken', 'and', 'irresponsible', 'mr', 'jone

In [16]:
len(text)

4167

### Making vocabs

Before making the vocabs, let's remove all question marks and perios, etc, then turn everything to lowercase, and then simply split the text. 

In [17]:
# combine everything into one to make vocabs
word_list = list(set(" ".join(text).split()))
word2id = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}  #special tokens.

#create the word2id
for i, w in enumerate(word_list):
    word2id[w] = i + 4  #because 0-3 are already occupied
    id2word = {i: w for i, w in enumerate(word2id)}
    vocab_size = len(word2id)

#list of all tokens for whole text
token_list = list()
for sentence in text:
    arr = [word2id[word] for word in sentence.split()]
    print(arr)
    token_list.append(arr)

[6395, 6523, 5280, 6395, 15064, 6377, 5280, 8168, 13541, 2545, 5280, 9176, 6377, 5280, 2998, 5840, 12436, 3478, 7017, 9258, 11210, 5280, 15798, 12515, 10262, 4391, 7510, 5280, 9176, 12436, 1387, 4804, 9372, 7286, 10087]
[8458, 7616, 8680, 10808, 10301, 3486, 5394, 4391, 16530, 14178, 4397, 4391, 12741, 17627, 5587, 10050, 12436, 14121]
[5280, 9176, 2663, 4391, 3370, 5280, 9160, 4391, 2275, 16723, 16333, 7119, 5280, 13541, 14352, 1735, 11520, 11596]
[16083, 8186, 9752, 4377, 7286, 7890, 5280, 5337, 11389, 7286, 5655, 2479, 14822, 9176, 509, 12083]
[5394, 10938, 12515, 6441, 5280, 9176, 17280, 4391, 8693, 17688, 16203, 15593, 4391, 5280, 2998, 5042, 13930]
[5280, 2285, 1602, 2941, 12515, 7525, 7286, 4147, 4391, 8236, 14773, 14275, 17688, 7858, 553, 5840, 13401, 7557, 13675]
[2627, 2011, 5280, 10246, 7119, 5280, 2998, 11750, 4391, 13371, 895, 108]
[2627, 4391, 5394, 3090, 5840, 2019]
[8458, 5394, 13723, 17627, 6435, 12515, 10524, 12436, 14703, 2627, 17109, 17627, 11750, 10215, 5394, 7790,

In [18]:
sentences[:5]

[ Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'.,
 When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy.,
 The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm".,
 They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal".,
 Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly.]

In [19]:
type(sentences[0])

spacy.tokens.span.Span

In [20]:
text[:5]

[" old major, the old boar on the manor farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'beasts of england'.",
 'when major dies, two young pigs, snowball and napoleon, assume command and turn his dream into a philosophy.',
 'the animals revolt and drive the drunken and irresponsible mr jones from the farm, renaming it "animal farm".',
 'they adopt seven commandments of animal-ism, the most important of which is, "all animals are equal".',
 'snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly.']

In [21]:
#take a look at sentences
sentences

[ Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'.,
 When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy.,
 The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm".,
 They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal".,
 Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly.,
 The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health.,
 Napoleon takes the pups from the farm dogs and trains them privately.,
 Napoleon and Snowball struggle for leadership.,
 When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares him

In [22]:
#take a look at token_list
token_list

[[6395,
  6523,
  5280,
  6395,
  15064,
  6377,
  5280,
  8168,
  13541,
  2545,
  5280,
  9176,
  6377,
  5280,
  2998,
  5840,
  12436,
  3478,
  7017,
  9258,
  11210,
  5280,
  15798,
  12515,
  10262,
  4391,
  7510,
  5280,
  9176,
  12436,
  1387,
  4804,
  9372,
  7286,
  10087],
 [8458,
  7616,
  8680,
  10808,
  10301,
  3486,
  5394,
  4391,
  16530,
  14178,
  4397,
  4391,
  12741,
  17627,
  5587,
  10050,
  12436,
  14121],
 [5280,
  9176,
  2663,
  4391,
  3370,
  5280,
  9160,
  4391,
  2275,
  16723,
  16333,
  7119,
  5280,
  13541,
  14352,
  1735,
  11520,
  11596],
 [16083,
  8186,
  9752,
  4377,
  7286,
  7890,
  5280,
  5337,
  11389,
  7286,
  5655,
  2479,
  14822,
  9176,
  509,
  12083],
 [5394,
  10938,
  12515,
  6441,
  5280,
  9176,
  17280,
  4391,
  8693,
  17688,
  16203,
  15593,
  4391,
  5280,
  2998,
  5042,
  13930],
 [5280,
  2285,
  1602,
  2941,
  12515,
  7525,
  7286,
  4147,
  4391,
  8236,
  14773,
  14275,
  17688,
  7858,
  553,
  5840

In [23]:
#testing one sentence
for tokens in token_list[0]:
    print(id2word[tokens])

old
major,
the
old
boar
on
the
manor
farm,
calls
the
animals
on
the
farm
for
a
meeting,
where
he
compares
the
humans
to
parasites
and
teaches
the
animals
a
revolutionary
song,
'beasts
of
england'.


## 2. Data loader

We gonna make dataloader.  Inside here, we need to make two types of embeddings: **token embedding** and **segment embedding**

1. **Token embedding** - Given “The cat is walking. The dog is barking”, we add [CLS] and [SEP] >> “[CLS] the cat is walking [SEP] the dog is barking”. 

2. **Segment embedding**
A segment embedding separates two sentences, i.e., [0 0 0 0 1 1 1 1 ]

3. **Masking**
As mentioned in the original paper, BERT randomly assigns masks to 15% of the sequence. In this 15%, 80% is replaced with masks, while 10% is replaced with random tokens, and the rest 10% is left as is.  Here we specified `max_pred` 

4. **Padding**
Once we mask, we will add padding. For simplicity, here we padded until some specified `max_len`. 

Note:  `positive` and `negative` are just simply counts to keep track of the batch size.  `positive` refers to two sentences that are really next to one another.

In [24]:
batch_size = 6
max_mask   = 5  # max masked tokens when 15% exceed, it will only be max_pred
# max_len    = max_word_in_sentence*2+3 # 205; maximum of length to be padded; max_len <= max_sentence_len * 2 + 3
max_len    = 512

In [25]:
max_sentence_len = 0
for i in range(len(token_list)):
    max_sentence_len = max(max_sentence_len, len(token_list[i]))
    print(len(token_list[i]))
max_sentence_len



35
18
18
16
17
19
12
6
20
22
16
13
10
24
23
23
19
20
16
13
19
20
17
20
32
21
15
14
21
28
14
20
15
38
19
17
36
5
8
12
6
8
6
8
5
43
10
2
9
1
1
7
1
1
9
26
6
39
18
17
37
29
26
32
28
34
19
20
6
34
10
15
41
17
20
31
20
25
39
22
37
30
31
25
20
12
32
14
17
34
25
16
30
22
23
23
22
23
21
19
10
19
13
28
25
22
21
16
23
20
17
12
9
23
21
5
15
27
27
25
14
32
27
19
23
41
14
7
13
21
22
20
13
11
18
28
24
23
37
15
30
19
10
16
13
28
18
12
14
22
9
17
8
14
4
17
6
22
10
3
7
20
20
14
17
25
7
30
16
9
13
16
22
16
21
20
76
2
14
2
19
23
27
38
2
17
19
22
11
32
17
2
26
15
2
35
17
18
8
20
14
2
38
15
25
12
20
27
20
51
2
10
23
2
11
21
19
21
15
22
18
22
39
101
2
20
2
27
32
24
2
30
23
17
2
23
28
18
27
35
23
25
24
2
17
24
92
4
28
8
20
29
42
18
2
17
51
2
27
14
37
2
15
25
23
14
34
28
19
40
48
18
31
28
45
2
35
33
8
18
16
34
13
16
30
11
9
12
27
25
2
15
14
28
13
16
35
19
17
17
21
24
24
27
30
26
19
31
38
23
18
26
24
28
22
17
18
24
14
25
28
7
32
30
28
30
0
48
14
25
43
24
24
48
28
21
15
6
9
8
20
16
26
8
22
33
21
24
7
10
20
15
9


101

In [26]:
batch = []
positive = negative = 0  #count of batch size;  we want to have half batch that are positive pairs (i.e., next sentence pairs)
while positive != batch_size/2 or negative != batch_size/2:
    
    #randomly choose two sentence so we can put [SEP]
    tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))

    #retrieve the two sentences
    tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

    #1. token embedding - append CLS and SEP
    input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]

    #2. segment embedding - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
    segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

    #3. mask language modeling
    #masked 15%, but should be at least 1 but does not exceed max_mask
    n_pred =  min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
    #get the pos that excludes CLS and SEP and shuffle them
    cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
    shuffle(cand_maked_pos)
    masked_tokens, masked_pos = [], []
    #simply loop and change the input_ids to [MASK]
    for pos in cand_maked_pos[:n_pred]:
        masked_pos.append(pos)  #remember the position
        masked_tokens.append(input_ids[pos]) #remember the tokens
        #80% replace with a [MASK], but 10% will replace with a random token
        if random() < 0.1:  # 10%
            index = randint(0, vocab_size - 1) # random index in vocabulary
            input_ids[pos] = word2id[id2word[index]] # replace
        elif random() < 0.9:  # 80%
            input_ids[pos] = word2id['[MASK]'] # make mask
        else:  #10% do nothing
            pass

    # pad the input_ids and segment ids until the max len
    n_pad = max_len - len(input_ids)   # max_len <= max_sentence_len * 2 + 3
    input_ids.extend([0] * n_pad)
    segment_ids.extend([0] * n_pad)

    # pad the masked_tokens and masked_pos to make sure the lenth is max_mask
    if max_mask > n_pred:
        n_pad = max_mask - n_pred
        masked_tokens.extend([0] * n_pad)
        masked_pos.extend([0] * n_pad)

    #check if first sentence is really comes before the second sentence
    #also make sure positive is exactly half the batch size
    if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
        batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
        positive += 1
    elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
        batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
        negative += 1

In [27]:
def make_batch():
    batch = []
    positive = negative = 0  #count of batch size;  we want to have half batch that are positive pairs (i.e., next sentence pairs)
    while positive != batch_size/2 or negative != batch_size/2:
        
        #randomly choose two sentence so we can put [SEP]
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))
        #retrieve the two sentences
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

        #1. token embedding - append CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]

        #2. segment embedding - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        #3. mask language modeling
        #masked 15%, but should be at least 1 but does not exceed max_mask
        n_pred =  min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get the pos that excludes CLS and SEP and shuffle them
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and change the input_ids to [MASK]
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)  #remember the position
            masked_tokens.append(input_ids[pos]) #remember the tokens
            #80% replace with a [MASK], but 10% will replace with a random token
            if random() < 0.1:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word2id[id2word[index]] # replace
            elif random() < 0.9:  # 80%
                input_ids[pos] = word2id['[MASK]'] # make mask
            else:  #10% do nothing
                pass

        # pad the input_ids and segment ids until the max len
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # pad the masked_tokens and masked_pos to make sure the lenth is max_mask
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        #check if first sentence is really comes before the second sentence
        #also make sure positive is exactly half the batch size
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
            
    return batch

In [28]:
batch = make_batch()

In [29]:
#len of batch
len(batch)

6

In [30]:
len('. '.join(text))

528491

In [31]:
input_ids

[1,
 10556,
 2569,
 12515,
 9858,
 2488,
 2941,
 16240,
 12436,
 9780,
 12515,
 14347,
 12436,
 8720,
 147,
 3536,
 5280,
 13070,
 5655,
 16240,
 17393,
 13435,
 13231,
 13910,
 7557,
 9322,
 15969,
 10351,
 5250,
 6470,
 17737,
 16240,
 12436,
 6969,
 2702,
 5161,
 6486,
 568,
 13629,
 12436,
 7915,
 3,
 16083,
 9960,
 10556,
 17107,
 16177,
 10820,
 2569,
 12515,
 6304,
 3976,
 7286,
 1817,
 14509,
 2,
 5250,
 3,
 17627,
 4397,
 3976,
 7286,
 5280,
 1076,
 14671,
 3,
 5055,
 694,
 3,
 14960,
 849,
 6377,
 16875,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [32]:
#we can deconstruct using map and zip
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext.shape

(torch.Size([6, 512]),
 torch.Size([6, 512]),
 torch.Size([6, 5]),
 torch.Size([6, 5]),
 torch.Size([6]))

## 3. Model

Recall that BERT only uses the encoder.

BERT has the following components:

- Embedding layers
- Attention Mask
- Encoder layer
- Multi-head attention
- Scaled dot product attention
- Position-wise feed-forward network
- BERT (assembling all the components)

## 3.1 Embedding

Here we simply generate the positional embedding, and sum the token embedding, positional embedding, and segment embedding together.

<img src = "figures/BERT_embed.png" width=500>

In [33]:
input_ids.shape

torch.Size([6, 512])

In [34]:
# for a in range(len(batch)):
#     for b in range(len(batch[a])):
#         try:
#             print(len(batch[a][b]))
#         except:
#             print(batch[a][b])

In [35]:
batch[0][4]

False

In [36]:
len(raw_text)

100

In [37]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        ### the error is in the next line

        # global x_fd, seg_fd, pos_fd
        # x_fd = x
        # seg_fd = seg
        # pos_fd = pos

        # print(x_fd.max(), pos_fd.max(), seg_fd.max())
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

## 3.2 Attention mask

In [38]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

### Testing the attention mask

In [39]:
print(get_attn_pad_mask(input_ids, input_ids).shape)

torch.Size([6, 512, 512])


## 3.3 Encoder

The encoder has two main components: 

- Multi-head Attention
- Position-wise feed-forward network

First let's make the wrapper called `EncoderLayer`

In [40]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn       = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

Let's define the scaled dot attention, to be used inside the multihead attention

In [41]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

Let's define the parameters first

In [42]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

Here is the Multiheadattention.

In [43]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]


Here is the PoswiseFeedForwardNet.

In [44]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))


## 3.4 Putting them together

In [45]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp

## 4. Training

In [46]:
max_len

512

In [47]:
device

device(type='cuda')

In [48]:
from model import bert

In [58]:
import importlib
importlib.reload(bert);

In [59]:
max_len

512

In [60]:
device

device(type='cuda')

In [61]:
num_epoch = 500
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = bert.BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

print('ok')
print(device)

ok
cuda


In [62]:
print(input_ids.shape)
print(segment_ids.shape)
print(masked_tokens.shape)
print(masked_pos.shape)
print(isNext.shape)

torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 5])
torch.Size([6, 5])
torch.Size([6])


In [63]:
vocab_size

17751

In [64]:
max(id2word.keys())

17750

In [65]:
max_len

512

In [66]:
for epoch in range(num_epoch): 
    optimizer.zero_grad()
    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)    
    #logits_lm: (bs, max_mask, vocab_size) ==> (6, 5, 34)
    #logits_nsp: (bs, yes/no) ==> (6, 2)

    #1. mlm loss
    #logits_lm.transpose: (bs, vocab_size, max_mask) vs. masked_tokens: (bs, max_mask)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    #2. nsp loss
    #logits_nsp: (bs, 2) vs. isNext: (bs, )
    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification
    
    #3. combine loss
    loss = loss_lm + loss_nsp
    if epoch % 100 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

## 5. Inference

In [ ]:
input_ids.device

device(type='cuda', index=0)

In [ ]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[2]))

input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)


print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])

logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)
#logits_lm:  (1, max_mask, vocab_size) ==> (1, 5, 34)
#logits_nsp: (1, yes/no) ==> (1, 2)

#predict masked tokens
#max the probability along the vocab dim (2), [1] is the indices of the max, and [0] is the first value
logits_lm = logits_lm.to('cpu')
logits_lm = logits_lm.data.max(2)[1][0].data.numpy() 
#note that zero is padding we add to the masked_tokens
print('masked tokens (words) : ',[id2word[pos.item()] for pos in masked_tokens[0]])
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0]])
print('masked tokens (words) : ',[id2word[pos.item()] for pos in logits_lm])
print('predict masked tokens list : ', [pos for pos in logits_lm])

#predict nsp
logits_nsp = logits_nsp.to('cpu')
logits_nsp = logits_nsp.data.max(1)[1][0].data.numpy()
print(logits_nsp)
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_nsp else False)

['[CLS]', 'the', 'resulting', 'images', '[MASK]', 'during', 'its', 'rapid', 'flyby', 'reveal', 'that', 'rama', 'is', 'a', 'perfect', '[MASK]', 'in', 'diameter', 'and', 'long,', 'made', 'of', 'a', '[MASK]', 'featureless', 'material,', 'making', 'this', "humankind's", 'first', '[MASK]', 'with', 'an', 'alien', 'space', 'ship.', '[SEP]', 'sun;', 'argues', 'that', 'this', 'is', 'part', 'of', 'a', 'trend', 'toward', 'targeting', 'younger', 'and', 'younger', 'consumers.', '[SEP]']
masked tokens (words) :  ['encounter', 'completely', 'klein', 'taken', 'cylinder,']
masked tokens list :  [11555, 10393, 14199, 15219, 1958]
masked tokens (words) :  ['[PAD]', '[PAD]', 'to', '[PAD]', '[PAD]']
predict masked tokens list :  [0, 0, 11570, 0, 0]
1
isNext :  False
predict isNext :  True


In [ ]:
torch.save(model, './model/BERT4.pt')
torch.save(model.state_dict(), './model/BERT4.pth')

In [ ]:
pickle.dump(word2id, open('./model/elements/word2id.pkl', 'wb'))
pickle.dump(id2word, open('./model/elements/id2word.pkl', 'wb'))

